### 识别手写数字算法

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# 定义数据变换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为 Tensor
    transforms.Normalize((0.5,), (0.5,))  # 归一化到 [-1, 1]
])

# 下载和加载训练集
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 下载和加载测试集
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# 检查数据集
print("Training set size:", len(trainset))
print("Testing set size:", len(testset))

# 检查 DataLoader
for images, labels in trainloader:
    print("Batch shape:", images.shape)
    print("Labels shape:", labels.shape)
    break  # 只检查第一个批次

Training set size: 60000
Testing set size: 10000
Batch shape: torch.Size([64, 1, 28, 28])
Labels shape: torch.Size([64])


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.5.1
12.4
True
NVIDIA GeForce GTX 1660 SUPER


In [1]:
import torch

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 定义数据变换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为 Tensor
    transforms.Normalize((0.5,), (0.5,))  # 归一化到 [-1, 1]
])

# 使用gpu设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 下载和加载训练集
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 下载和加载测试集
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
# 定义卷积神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 一次卷积
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # 第二次卷积
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # 池化
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 线性化
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # 一次卷积、池化
        x = self.pool(self.relu(self.conv1(x)))
        # 第二次卷积、池化
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # 展平特征图
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# 初始化模型
net = Net()
net.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # 将数据移动到设备
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / (i+1):.4f}")
# 评估模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 将数据移动到设备
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%")


Using device: cuda
Epoch [1/10], Loss: 0.2775
Epoch [2/10], Loss: 0.0987
Epoch [3/10], Loss: 0.0744
Epoch [4/10], Loss: 0.0600
Epoch [5/10], Loss: 0.0483
Epoch [6/10], Loss: 0.0402
Epoch [7/10], Loss: 0.0369
Epoch [8/10], Loss: 0.0308
Epoch [9/10], Loss: 0.0305
Epoch [10/10], Loss: 0.0270
Accuracy of the network on the 10000 test images: 98.63%


In [2]:
# 获取每一层的参数
for name, param in net.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}")
        print(f"Shape: {param.data.shape}")
        print(f"Weights: {param.data}\n")

Layer: conv1.weight
Shape: torch.Size([32, 1, 3, 3])
Weights: tensor([[[[-0.2426, -0.4322,  0.3620],
          [-0.3992,  0.1563,  0.3964],
          [-0.0538,  0.4213,  0.0371]]],


        [[[ 0.2113, -0.0259, -0.2114],
          [ 0.4367, -0.1806, -0.1794],
          [ 0.4631,  0.2454, -0.3876]]],


        [[[-0.2977,  0.0518,  0.2209],
          [-0.3533,  0.1516,  0.1408],
          [-0.4256,  0.3252,  0.3500]]],


        [[[ 0.7013, -0.3188, -0.1568],
          [ 0.1043, -0.4721,  0.0303],
          [ 0.2311, -0.0786,  0.1505]]],


        [[[-0.3197, -0.3203, -0.3587],
          [ 0.2040,  0.0571,  0.0705],
          [ 0.2878,  0.2681,  0.2020]]],


        [[[-0.1313, -0.0807,  0.0552],
          [ 0.4447,  0.4402,  0.3964],
          [-0.2656, -0.3703, -0.3862]]],


        [[[-0.0400, -0.3263, -0.0758],
          [ 0.1621, -0.2617, -0.2336],
          [ 0.6032, -0.1176,  0.2780]]],


        [[[-0.2402, -0.0969, -0.2023],
          [ 0.1145, -0.2929, -0.0758],
          [-0

In [5]:
from PIL import Image
import matplotlib.pyplot as plt


# 使用gpu设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def predict_image(image_path, model, transform):
    # 加载图片
    image = Image.open(image_path).convert('L')  # 转换为灰度图像
    image = transform(image).unsqueeze(0)  # 转换为 Tensor 并增加 batch 维度
    image.to(device)

    # 使用模型进行预测
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    
    return predicted.item()

# 指定图片路径
image_path = 'D:\code\python-learning\mnist\generic\8.png'

# 进行预测
predicted_digit = predict_image(image_path, net, transform)
print(f"Predicted digit: {predicted_digit}")

Using device: cuda


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper_CUDA___slow_conv2d_forward)